# Laboratorio 2 - from definition to sense

I comuni dizionari a cui siamo abituati partono dalle parole, ovvero dalla forma, per arrivare al contenuto.
Esistono alcuni tipi di dizionario chiamati dizionari analogici che funzionano ”al contrario”, ovvero non si ricerca per parola ma per definizione. Questo tipo di ricerca viene chiamata ricerca onomasiologica, ovvero si parte dal contenuto per arrivare alla forma. Proprio su questo si basa la seconda esercitazione.
Sempre partendo dai dati sulle definizioni, si richiede di provare a costruire un sistema che utilizzi la molteplicità delle definizioni per risalire al termine "target" in maniera automatica.
Non si richiede di "indovinare" ogni termine, ma di avvicinarsi (almeno semanticamente) alla risposta.
Provare più soluzioni, includendo meccanismi di filtro delle definizioni (ad es. escludendo quelle meno informative o con caratteristiche particolari), di ricerca nell'albero tassonomico di WordNet (provando a partire da candidati "genus", secondo il principio Genus-Differentia), ecc.

## Import delle librerie

In [1]:
base_folder = './data'
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from statistics import mean
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from operator import itemgetter
from nltk.corpus import wordnet as wn

[nltk_data] Downloading package stopwords to /Users/mario/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/mario/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Lettura dell'input

Come per l'esercizio 1, leggiamo il file *tsv* per salvare le definizioni associate alle parole. Le definizioni sono raccolte in un dizionario le cui chiavi sono rappresentate dai termini.

Per ottenere definizioni quanto più informative vengono rimosse le stopwords e viene effettuata l'operazione di lemmatizzazione. In questo modo è possibile salvare l'insieme delle parole *content* come definizione.

In [4]:
lemmatizer = WordNetLemmatizer()

In [5]:
def remove_stopwords(phrase):
    phrase = {lemmatizer.lemmatize(p) for p in phrase}
    punct = string.punctuation
    for p in punct:
        phrase = {item.replace(p, '') for item in phrase}
    phrase = {item.replace('\'s', '') for item in phrase}
    stop = stopwords.words('english')
    return {t for t in phrase if t not in stop}

In [6]:
def read_data(path):
    with open(path) as f:
        lines = f.readlines()
    lines = [x.strip().split('\t') for x in lines]

    defs = {}
    for i, e in enumerate(lines[0][1:]):
        defs[e] = [remove_stopwords(s[i+1].lower().split()) for s in lines[1:]]
    return defs

In [7]:
definitions = read_data(f'{base_folder}/TLN-definitions-23.tsv')
display(definitions)

{'door': [{'closing',
   'construction',
   'divide',
   'passage',
   'rooms',
   'temporarily',
   'two',
   'used'},
  {'closed', 'opened', 'opening'},
  {'closing',
   'divide',
   'door',
   'enter',
   'get',
   'hole',
   'let',
   'object',
   'open',
   'people',
   'room',
   'two',
   'wall'},
  {'access', 'another', 'area', 'one', 'usable'},
  {'access', 'allows', 'area', 'delimits', 'structure'},
  {'block', 'moved', 'object', 'pas', 'passage', 'used'},
  {'also',
   'assembled',
   'body',
   'building',
   'consists',
   'handle',
   'hinge',
   'historically',
   'iron',
   'lock',
   'locked',
   'made',
   'main',
   'materials',
   'mean',
   'moving',
   'object',
   'opened',
   'pushed',
   'requires',
   'room',
   'rotates',
   'separate',
   'sometimes',
   'unlock',
   'used',
   'wood'},
  {'closed', 'go', 'object', 'opened', 'room', 'separate', 'used', 'wall'},
  {'access', 'another', 'opened', 'order', 'place', 'something'},
  {'access', 'room'},
  {'access

## Ricerca del Genus

Secondo la teoria *Genus-Diferentia*, la parola più frequente nelle definizioni è molto probabilmente collegata al termine stesso che stiamo cercando.

Per questo motivo, il metodo *find_most_frequent_words* calcola la frequenza con cui compaiono le parole in una stringa e restituisce due array ordinati, il primo contenente le parole più frequenti e il secondo contenente le frequenze stesse.

In [8]:
def find_most_frequent_words(text):
    text = [w.strip() for w in text.split()]
    words_freq = {(w, text.count(w)) for w in text}
    ordered = sorted(words_freq, key=itemgetter(1), reverse=True)
    return [o[0] for o in ordered], [o[1] for o in ordered]

## Calcolo della similarità su WordNet

Il seguente metodo, dato un synset di WordNet e una lista di definizioni, calcola la similarità media tra le definizioni annotate e la stringa contesto ottenuta dall'unione degli esempi presenti su WordNet e dalla definizione online.

In [9]:
def get_similarity(s, defs):
    synset_words = set()
    synset_words = synset_words.union(s.definition().split())
    for ex in s.examples():
        synset_words = synset_words.union(ex.split())
    synset_words = remove_stopwords(synset_words)
    similarities = []
    for d in defs:
        similarities.append(len(synset_words.intersection(d)) / min(len(synset_words), len(d)))
    return mean(similarities)

## Ricerca nella gerarchia

Il metodo *wordnet_search* percorre tre livelli dell'albero di wordnet a partire da un termine, calcolando per ogni possibile senso la similarità e restituendo i sensi dal più probabile a quello più inverosimile.

La scelta di effettuare un numero di salti pari a tre è stata fatta per evitare una ricerca esaustiva nell'albero, che non sarebbe molto significativa per il task. Inoltre, da un'analisi svolta a mano si può verificare che tutti i termini che stiamo cercando distano al più due iperonimi dal giusto Genus estratto dal dataset.

In [10]:
def wordnet_search(t, defs):
    bests = []
    min_sim = 0
    for s1 in wn.synsets(t, pos='n'):
        sim = get_similarity(s1, defs)
        if sim > min_sim:
            min_sim = sim
            bests.append((s1, sim))
        for s2 in s1.hyponyms():
            sim = get_similarity(s2, defs)
            if sim > min_sim:
                min_sim = sim
                bests.append((s2, sim))
            for s3 in s2.hyponyms():
                sim = get_similarity(s3, defs)
                if sim > min_sim:
                    min_sim = sim
                    bests.append((s3, sim))
    return sorted(bests, key=itemgetter(1), reverse=True)

Di seguito, vengono elencati i cinque migliori sensi data la lista di definizioni. Per effettuare il confronto, viene stampato anche il termine ricercato.

Si può notare che, per i termini concreti, il senso corretto è tra i primi tre restituiti. Quindi, in questi casi, le definizioni sono state abbastanza precise per permettere di recuperare il senso associato.

Il synset relativo al termine *pain* non viene trovato, tuttavia si può notare come i sensi recuperati non siano semanticamente distanti dal target.

Situazione diversa, invece, per *blurriness*, termine per il quale i sensi restituiti non sono collegati se non tangenzialmente. Analizzando il dataset e le frequenza di comparsa delle parole, si deduce che l'unico termine che avrebbe potuto portare al senso *blurriness*, ovvero il Genus corretto, è *quality*, che però appare con una frequenza pari a uno, motivo per cui non è considerato un Genus candidato dall'algoritmo.

In [11]:
top_k = 5

In [12]:
for k, v in definitions.items():
    all_words = ''
    for words in v:
        all_words += ' ' + ' '.join(words)
    terms, freqs = find_most_frequent_words(all_words)[:top_k]
    candidates = {}
    for idx, term in enumerate(terms):
        if term in {'object', 'entity'}:
            continue
        for sense, similarity in wordnet_search(term, v):
            if sense not in candidates:
                candidates[(term, sense)] = similarity * freqs[idx] / mean(freqs)
            else:
                candidates[(term, sense)] += similarity * freqs[idx] / mean(freqs)
    print(k)
    for c in sorted(candidates.items(), key=itemgetter(1), reverse=True)[:top_k]:
        print(f'\t{str(c[0])} - {round(c[1], 3)}')

door
	('room', Synset('dining_room.n.01')) - 1.207
	('access', Synset('doorway.n.01')) - 0.981
	('room', Synset('bedroom.n.01')) - 0.946
	('room', Synset('room.n.01')) - 0.908
	('access', Synset('entrance.n.01')) - 0.533
ladybug
	('insect', Synset('ephemerid.n.01')) - 3.709
	('insect', Synset('ladybug.n.01')) - 2.338
	('red', Synset('wine.n.02')) - 2.132
	('black', Synset('coal_black.n.01')) - 2.086
	('insect', Synset('ground_beetle.n.01')) - 1.964
pain
	('feeling', Synset('feeling.n.04')) - 2.195
	('feeling', Synset('suffering.n.04')) - 2.026
	('feeling', Synset('emotion.n.01')) - 1.754
	('feeling', Synset('ambivalence.n.01')) - 1.378
	('feeling', Synset('feeling.n.01')) - 1.294
blurriness
	('image', Synset('memory_picture.n.01')) - 0.383
	('vision', Synset('sight.n.03')) - 0.334
	('condition', Synset('emmetropia.n.01')) - 0.286
	('image', Synset('memory_image.n.01')) - 0.272
	('focus', Synset('particularism.n.01')) - 0.261
